In [48]:
import pandas as pd

import glob                        
import pandas as pd                
import xml.etree.ElementTree as ET 
from datetime import datetime
import requests

import warnings
warnings.simplefilter(action='ignore')

# Unzip Files

In [2]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


# About the Data 

The file `dealership_data` contains CSV, JSON, and XML files for used car data which contain features named `car_model`, `year_of_manufacture`, `price`, and `fuel`.


# Set Paths

In [3]:
tmpfile = 'dealership_temp.tmp'
logfile = 'dealership_log.txt'
targetfile = 'dealership_transformed_data.csv'

# Extract 

## Extract csv

In [4]:
def extract_csv(data):
    df_csv = pd.read_csv(data)
    return df_csv

## Extract Json

In [7]:
def extract_json(data):
    df_json = pd.read_json(data, lines=True)
    return df_json

## Extract Xml

In [23]:
def extract_from_xml(data):
    df_xml = pd.DataFrame(columns=["car_model", "year_of_manufacture", "price", "fuel"])
    tree = ET.parse(data)
    root = tree.getroot()
    for car in root:
        car_model = car.find("car_model").text
        year_of_manufacture = float(car.find("year_of_manufacture").text)
        price = float(car.find("price").text)
        fuel = str(car.find("fuel").text)
        df_xml = df_xml.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return df_xml

## Extract Function

In [10]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

# Transform

## Round the Price columns to 2 decimal places

In [41]:
def transform_two_decimal(data):
    data.price = data.price.apply(lambda x: round(x, 2))
    data.year_of_manufacture = data.year_of_manufacture.apply(lambda x: int(x))
    return data

# Load

In [14]:
def load(targetfile, data):
    data.to_csv(targetfile)

# Logging

In [16]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

# Running ETL Process

In [44]:
log('ETL started')

log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

log("Transform phase Started")
transformed_data = transform_two_decimal(extracted_data)
log("Transform phase Ended")
transformed_data 

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel
...,...,...,...,...
85,etios liva,2014,7089.55,Diesel
86,innova,2017,29477.61,Petrol
87,fortuner,2010,13805.97,Diesel
88,corolla altis,2011,6492.54,Petrol


# Data Engineer Web Scraping

In [45]:
link = 'https://en.wikipedia.org/wiki/List_of_largest_banks'

# Peer Review Assignment - Data Engineer - Extract API Data

In [70]:
# Write your code here
website = 'https://exchangeratesapi.io'
url = "https://api.apilayer.com/exchangerates_data/latest?base=EUR&apikey=fJbgOoOeI0uJgk3yvRhoLLk91KdGWsw7" #Make sure to change ******* to your API key.

payload = {}
headers= {
  "apikey": "fJbgOoOeI0uJgk3yvRhoLLk91KdGWsw7"
}

response = requests.request("GET", url, headers=headers, data = payload)

status_code = response.status_code
result = response.text
print(result[:100])

{
    "success": true,
    "timestamp": 1676733243,
    "base": "EUR",
    "date": "2023-02-18",
   


In [67]:
import json
result_json = response.json()
print(type(result_json))
result_json

<class 'dict'>


{'success': True,
 'timestamp': 1676703604,
 'base': 'EUR',
 'date': '2023-02-18',
 'rates': {'AED': 3.937634,
  'AFN': 95.411639,
  'ALL': 116.048378,
  'AMD': 421.647183,
  'ANG': 1.932588,
  'AOA': 542.990728,
  'ARS': 206.499627,
  'AUD': 1.558424,
  'AWG': 1.929666,
  'AZN': 1.826718,
  'BAM': 1.973858,
  'BBD': 2.165125,
  'BDT': 113.45455,
  'BGN': 1.960316,
  'BHD': 0.403996,
  'BIF': 2219.115448,
  'BMD': 1.072036,
  'BND': 1.437308,
  'BOB': 7.409975,
  'BRL': 5.540328,
  'BSD': 1.072369,
  'BTC': 4.3522112e-05,
  'BTN': 88.79537,
  'BWP': 14.184423,
  'BYN': 2.706722,
  'BYR': 21011.914385,
  'BZD': 2.161492,
  'CAD': 1.444516,
  'CDF': 2194.458997,
  'CHF': 0.991571,
  'CLF': 0.030668,
  'CLP': 846.212395,
  'CNY': 7.362322,
  'COP': 5252.882113,
  'CRC': 599.703408,
  'CUC': 1.072036,
  'CUP': 28.408966,
  'CVE': 111.08982,
  'CZK': 23.755796,
  'DJF': 190.52274,
  'DKK': 7.463629,
  'DOP': 60.013022,
  'DZD': 146.428102,
  'EGP': 32.673539,
  'ERN': 16.080547,
  'ETB': 57

In [68]:
# Turn the data into a dataframe
df = pd.DataFrame(result_json)

# Drop unnescessary columns
df = df.drop(columns=['success', 'timestamp', 'base', 'date'])
df.head()

,rates
AED,3.937634
AFN,95.411639
ALL,116.048378
AMD,421.647183
ANG,1.932588


In [69]:
# Save the Dataframe
df.to_csv('exchange_rates_1.csv')

# Peer Review Assignment - Data Engineer - ETL

## Imports

Import any additional libraries you may need here.

In [72]:
import glob
import pandas as pd
import json
from datetime import datetime

## Extract

### Json Extract
This function will extract JSON files.

In [73]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

In [90]:
columns=['Name','Market Cap (US$ Billion)']

def extract():
    # Write your code here
    extracted_data = pd.DataFrame(columns=columns)

    # process json file
    extracted_data = extract_from_json('bank_market_cap_1.json')
    
    return extracted_data

In [80]:
df_rate = pd.read_csv('exchange_rates.csv')
df_rate.columns = ['symbol', 'rates']
df_rate.head()

,symbol,rates
0,AUD,1.297088
1,BGN,1.608653
2,BRL,5.409196
3,CAD,1.271426
4,CHF,0.886083


In [81]:
df_rate.loc[df_rate.symbol == 'GBP']

,symbol,rates
9,GBP,0.732398


In [98]:
exchange_rate = float(df_rate.loc[df_rate.symbol == 'GBP'].rates.values)
exchange_rate

0.7323984208000001

In [83]:
def transform(data):
    # Write your code here
    data['Market Cap (US$ Billion)'] = data['Market Cap (US$ Billion)'].apply(lambda x : round((x * exchange_rate), 3))
    data.columns = ['Name', 'Market Cap (GBP$ Billion)']
    return data

In [84]:
def load(target_file, data):
    data.to_csv(target_file)

In [85]:
def log(message):
    # Write your code here
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile2.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

In [91]:
log('ETL started')

log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
print(extracted_data.head())

log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
print(transformed_data.head())

log("Load phase Started")
load('target.csv', transformed_data)
log("Load phase Ended")

                                      Name  Market Cap (US$ Billion)
0                           JPMorgan Chase                   390.934
1  Industrial and Commercial Bank of China                   345.214
2                          Bank of America                   325.331
3                              Wells Fargo                   308.013
4                  China Construction Bank                   257.399
                                      Name  Market Cap (GBP$ Billion)
0                           JPMorgan Chase                    286.319
1  Industrial and Commercial Bank of China                    252.834
2                          Bank of America                    238.272
3                              Wells Fargo                    225.588
4                  China Construction Bank                    188.519
